<a href="https://colab.research.google.com/github/uhdang/mlim_group_3_final_assignment/blob/main/project/train_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO:

- Downsample training set (reduce method to delete irrelevant rows)
- maybe PCA()

# Google Colab Runtime
Run if using Google Colab

In [1]:
# # # import pandas as pd
# # # 
# # import numpy as np# 
# # 
# import o# s# 
# 
# from google.colab import dri# v# e# 
# 
# import # s# y# s#
# drive.mount('/content/dri# v# e# '# )
# sys.path.append('/content/drive/My Drive/My Dri# v# e# '# )
#
# from dataloader import Dat# a# l# o# ader
# # L# o# a# d#  Data
# path = "/content/drive/MyDrive/My Drive/"

# Local Runtime
Run if using notebook locally

In [2]:
import pandas as pd
import os
import optuna.integration.lightgbm as lightgb
import pickle
import sklearn

from sklearn.metrics import roc_auc_score
from sklearn.utils import resample
from tqdm import tqdm

pd.options.mode.chained_assignment = None  # default='warn'

from dataloader import Dataloader
from dataloader import create_combined_dict


# Load Data
path = os.getcwd() + "/../data/"

# Load Data

In [3]:
# load data which creates baskets and coupons
data = Dataloader(path)

In [4]:
# Create Categories before CV as they don't change and consume time otherwise
data.create_category_table(10000)

In [5]:
%%capture
weeks = [86, 87, 88, 89]
shopper_list = list(range(2000))
shopper_chunks = [shopper_list[i:i + 100] for i in range(0, len(shopper_list), 100)]

cv_dict = {
    'X_train': list(),
    'y_train': list(),
    'X_test': list(),
    'y_test': list()
}

In [6]:
for i, week in enumerate(weeks):
  print(f"week: {week}")

  X_train_list = list()
  y_train_list = list()
  X_test_list = list()
  y_test_list = list()

  for idx, shopper in enumerate(tqdm(shopper_chunks)):
      # print(f"shopper_chunk index: {idx}")

      # train-test-split
      data.train_test_split(week, shopper)

      # data add categories
      data.add_categories()

      # create features
      data.create_feature_dict()

      # combine everything
      if i == 0:
          X_train, y_train, X_test, y_test = data.make_featured_data()
          X_train_list.append(X_train)
          y_train_list.append(y_train)
          del X_train, y_train
      else:
          _, _, X_test, y_test = data.make_featured_data()

      X_test_list.append(X_test)
      y_test_list.append(y_test)
      del X_test, y_test

  cv_dict = create_combined_dict(X_train_list, y_train_list, X_test_list, y_test_list, cv_dict)

100%|██████████| 20/20 [49:15<00:00, 147.78s/it]


week: 86
week: 87
week: 88
week: 89


In [8]:
with open('cv_dict.pickle', 'wb') as handle:
    pickle.dump(cv_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [37]:
with open('dataloader.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

feature_fraction, val_score: inf:   0%|          | 0/7 [40:06<?, ?it/s]


# Hyperparameter Tuning

In [10]:
# %%capture
# !pip install optuna

In [3]:
sklearn.__version__

'2.2.0'

In [5]:
def downsample(df, y):
  df_target_coupon = df.loc[(y==1) | (df['coupon']=='Yes')]
  y_target_coupon = y.loc[(y==1) | (df['coupon']=='Yes')]
  df_down = df.loc[(y==0) & (df['coupon']=='No')]
  y_down = y.loc[(y==0) & (df["coupon"]=="No")]
  df_down, y_down = resample(
      df_down,
      y_down,
      replace=False,
      n_samples=df_target_coupon.shape[0],
      stratify=df_down['shopper']
  )
  df_all = pd.concat([df_target_coupon, df_down], ignore_index=True)
  y_all = pd.concat([y_target_coupon, y_down], ignore_index=True)

  return df_all, y_all

In [6]:
dtrain = lightgb.Dataset(cv_dict['X_train'][0], label=cv_dict['y_train'][0])
X_test0, y_test0 = downsample(cv_dict['X_test'][0], cv_dict['y_test'][0])
dval0 = lightgb.Dataset(X_test0, label=y_test0)
X_test1, y_test1 = downsample(cv_dict['X_test'][1], cv_dict['y_test'][1])
dval1 = lightgb.Dataset(X_test1, label=y_test1)
X_test2, y_test2 = downsample(cv_dict['X_test'][2], cv_dict['y_test'][2])
dval2 = lightgb.Dataset(X_test2, label=y_test2)
X_test3, y_test3 = downsample(cv_dict['X_test'][3], cv_dict['y_test'][3])
dval3 = lightgb.Dataset(X_test3, label=y_test3)

In [7]:

params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "verbosity": -1,
    "boosting_type": "gbdt",
}

lgb_clf = lightgb.train(
    params,
    dtrain,
    categorical_feature=['shopper', 'product', 'category', 'coupon', 'coupon_in_same_category'],
    valid_sets=[dval0, dval1, dval2],
    verbose_eval=100,
    early_stopping_rounds=100
)

[I 2021-03-20 23:48:20,066] A new study created in memory with name: no-name-4eeefd92-ea17-42df-bd3b-e16f6575899d
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]C:\Users\uh_da\anaconda3\envs\hu-mlim\lib\site-packages\lightgbm\basic.py:1161: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['category', 'coupon', 'coupon_in_same_category', 'product', 'shopper']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\uh_da\anaconda3\envs\hu-mlim\lib\site-packages\lightgbm\basic.py:725: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')
min_data_in_leaf, val_score: 0.253092: 100%|##########| 5/5 [09:21<00:00, 112.27s/it]


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.312626	valid_1's binary_logloss: 0.312025	valid_2's binary_logloss: 0.317081
Early stopping, best iteration is:
[30]	valid_0's binary_logloss: 0.257873	valid_1's binary_logloss: 0.258659	valid_2's binary_logloss: 0.259041
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.315205	valid_1's binary_logloss: 0.31444	valid_2's binary_logloss: 0.31956
Early stopping, best iteration is:
[30]	valid_0's binary_logloss: 0.25931	valid_1's binary_logloss: 0.260424	valid_2's binary_logloss: 0.260404
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.290617	valid_1's binary_logloss: 0.29108	valid_2's binary_logloss: 0.293652
Early stopping, best iteration is:
[32]	valid_0's binary_logloss: 0.253532	valid_1's binary_logloss: 0.253969	valid_2's binary_logloss: 0.254452
Training until validation scores don't improve 

In [22]:

# prediction = lgb_clf.predict(cv_dict['X_test'][3], num_iteration=lgb_clf.best_iteration)

array([0.01854805, 0.01983229, 0.01854805, ..., 0.01918106, 0.02008953,
       0.56972616])

In [3]:
# Loading from pickle - prediction
with open(os.getcwd() + "/../data/pickle/pred.pickle", 'rb') as f:
  prediction =  pickle.load(f)

In [4]:
# Load Data From Pickle

with open(os.getcwd() + "/../data/pickle/cv_dict_01.pickle", 'rb') as f:
  cv_dict =  pickle.load(f)

In [5]:
AUC = roc_auc_score(
    y_true = cv_dict['y_test'][3],
    y_score = prediction
)

In [6]:

# Loading from pickle - best_param
with open(os.getcwd() + "/../data/pickle/best_param.pickle", 'rb') as f:
  best_param =  pickle.load(f)


In [9]:
# best_params = lgb_clf.params
print("Best params:", best_param)
print("  Accuracy = {}".format(AUC))
print("  Params: ")
for key, value in best_param.items():
    print("    {}: {}".format(key, value))

Best params: {'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 5.024548095870188e-06, 'lambda_l2': 7.715874633155568, 'num_leaves': 37, 'feature_fraction': 0.4, 'bagging_fraction': 0.6190040709189294, 'bagging_freq': 5, 'min_child_samples': 20, 'categorical_column': [0, 1, 2, 105, 111]}
  Accuracy = 0.9424730857826296
  Params: 
    objective: binary
    metric: binary_logloss
    verbosity: -1
    boosting_type: gbdt
    feature_pre_filter: False
    lambda_l1: 5.024548095870188e-06
    lambda_l2: 7.715874633155568
    num_leaves: 37
    feature_fraction: 0.4
    bagging_fraction: 0.6190040709189294
    bagging_freq: 5
    min_child_samples: 20
    categorical_column: [0, 1, 2, 105, 111]


# Prediction and Revenue Optimization

# Playground

In [25]:
# train-test-split
data.train_test_split(89, list(range(50)))

# data add categories 
data.add_categories()
data.create_feature_dict()
X_train = data._make_full_table(data.baskets_train, data.coupons_train)
X_test = data._make_full_table(data.baskets_test, data.coupons_test)

X_train = data._merge_features(X_train, data.feature_dict)
X_test = data._merge_features(X_test, data.feature_dict)

In [26]:
X_train.head()